In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Install the vncorenlp python wrapper
!pip3 install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-08-05 07:53:52--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   111MB/s    in 0.2s    

2022-08-05 07:53:52 (111 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2022-08-05 07:53:52--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.co

In [4]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [5]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForQuestionAnswering
import torch

In [6]:
model_path = "/content/drive/MyDrive/IT4998/models/PhoBERT/PhoBERT-Question-Answering-June_29_2022_11h_37m_46s" 

In [7]:
config = AutoConfig.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, do_lower_case = False, use_fast=False)
model = AutoModelForQuestionAnswering.from_pretrained(model_path, config=config)

In [11]:
def predict_phobert(question, context):
    question, text = question, context
    question = " ".join(" ".join(sentence) for sentence in rdrsegmenter.tokenize(question))
    text = " ".join(" ".join(sentence) for sentence in rdrsegmenter.tokenize(text))

    inputs = tokenizer(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index + 1 : answer_end_index + 2]
    res = tokenizer.decode(predict_answer_tokens)

    res = res.replace("_", " ")
    return res

# RUN

In [12]:
question = """
Các yếu tố tạo nên chu kỳ Milankovitch có tác động như thế nào tới Trái Đất?
"""

text = """
Những biến đổi nhỏ về quỹ đạo Trái Đất gây ra những thay đổi về sự phân bố năng lượng mặt trời theo mùa trên bề mặt Trái Đất và cách nó được phân bố trên toàn cầu. Đó là những thay đổi rất nhỏ theo năng lượng mặt trời trung bình hàng năm trên một đơn vị diện tích; nhưng nó có thể gây biến đổi mạnh mẽ về sự phân bố các mùa và địa lý. Có 3 kiểu thay đổi quỹ đạo là thay đổi quỹ đạo lệch tâm của Trái Đất, thay đổi trục quay, và tiến động của trục Trái Đất. Kết hợp các yếu tố trên, chúng tạo ra các chu kỳ Milankovitch, là các yếu tố ảnh hưởng mạnh mẽ đến khí hậu và mối tương quan của chúng với các chu kỳ băng hà và gian băng, quan hệ của chúng với sự phát triển và thoái lui của Sahara, và đối với sự xuất hiện của chúng trong các địa tầng.
"""

In [13]:
print(predict_phobert(question, text))

ảnh hưởng mạnh mẽ đến khí hậu và mối tương quan của chúng với các chu kỳ băng hà và gian băng, quan hệ của chúng với sự phát triển và thoái lui của Sahara, và đối với sự xuất hiện của chúng trong các địa tầng
